# Dog Breed Identification: DLNN Final Project
Ian Battin & Liam Schmid




In this project, we are tackling the Kaggle Competition: [Dog Breed Identification](https://www.kaggle.com/c/dog-breed-identification/).

In this competition, you are given a dataset containing images of 120 different breeds of dogs. It is our job to classify images in the test set into these 120 different breeds.

Being an image based challenge, CNNs are the obvious choice. We will discuss the different strategies and optimizations we tried as well as provide all the code used at each step.

## Preparing the data
The data is provided in two folders - train and test - where each file is an image. There is a labels.csv file which maps an image filename to a label (breed).

To prepare our data for use, we needed to move all of the images in the training set into seperate folders grouped by breed: